In [ ]:
%load_ext autoreload
%autoreload 2

# Write weighted-actual name pairs for human review

In [ ]:
from src.data.filesystem import fopen
from src.data.utils import load_datasets, select_frequent_k
from src.models.utils import remove_padding

In [ ]:
given_surname = "given"
size = 200000
train_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-train.csv.gz"
test_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-test.csv.gz"
train_path_csv=f"s3://familysearch-names/processed/tree-hr-{given_surname}-train-{size}-weighted-actuals.csv"
test_path_csv=f"s3://familysearch-names/processed/tree-hr-{given_surname}-test-weighted-actuals.csv"

In [ ]:
[train, test] = load_datasets([train_path, test_path])

In [ ]:
input_names_train, weighted_actual_names_train, candidate_names_train = train
input_names_test, weighted_actual_names_test, candidate_names_test = test

In [ ]:
input_names_train, weighted_actual_names_train, candidate_names_train = \
    select_frequent_k(input_names_train,
                      weighted_actual_names_train,
                      candidate_names_train,
                      size)

In [ ]:
def write_csv(input_names, weighted_actual_names, path):
    lines = []
    for input_name, wans in zip(input_names, weighted_actual_names):
        # names don't have commas, so we can do this
        wans = ",".join([f"{remove_padding(name)}|{weight}" for name, weight, _ in wans])
        line = f"{remove_padding(input_name)},{wans}"
        lines.append(line)
    with fopen(path, "wt", encoding="utf-8") as f:
        f.write("\n".join(lines))

In [ ]:
write_csv(input_names_train, weighted_actual_names_train, train_path_csv)

In [ ]:
write_csv(input_names_test, weighted_actual_names_test, test_path_csv)